In [157]:
import simpy
from datetime import datetime as dt
import pandas as pd
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt

In [304]:
# ----------------------------------------------------------------------------!
def lognorm(mean, std):
    mu = np.log(mean ** 2 / np.sqrt(mean ** 2 + std ** 2))
    sigma = np.sqrt(np.log(1 + std ** 2 / mean ** 2))
    return st.lognorm(s=sigma, scale=np.exp(mu))


# ----------------------------------------------------------------------------!
class EventLog(object):
    def __init__(self, env: simpy.Environment, **kwargs):
        
        self.env = env
        self.log = {
            'Timestamp': [],
            'ID': [],
            'Description': [],
            'Activity state': [],
            'Stock': [],
            'Onboard': [],
            'Installed': [],
        }

    def entry(self, ID, description, activity_state, port, container, owf,
              **kwargs):
        self.log['Timestamp'].append(dt.fromtimestamp(self.env.now))
        self.log['ID'].append(ID)
        self.log['Description'].append(description)
        self.log['Activity state'].append(activity_state)
        self.log['Stock'].append(port.container.level)
        self.log['Onboard'].append(container.level)
        self.log['Installed'].append(owf.container.level)


# ----------------------------------------------------------------------------!
class Vessel(object):
    instances = []

    def __init__(self, env: simpy.Environment, ID: str, log: EventLog,
                 port: Site, owf:Site, capacity=5, **kwargs):
        self.env = env
        self.log = log
        self.ID = ID
        self.port = port
        self.container = simpy.Container(env=env, capacity=capacity, init=0)
        self.owf = owf

        # Manually adjust these!
        self.sail_to_length = lognorm(mean=24 * 3600, std=1200)
        self.sail_length = lognorm(mean=3600 * 1.9, std=1800)
        self.sail_between_length = lognorm(mean=3600 * 0.5, std=300)
        self.load_length = lognorm(mean=3600 * 3.7, std=600)
        self.install_length = lognorm(mean=3600 * (33.2), std=3600)

        # Add new class object to the list of instances
        Vessel.instances.append(self)

    def execute_actions(self):
        env = self.env

        # Navigate from offshore to port
        sail = Navigate(duration=self.sail_to_length.rvs(), 
                        description='Navigate from offshore to site',
                        **self.__dict__)
        yield env.process(sail.execute())

        while True:
            # Check whether it is necessary to perform activities
            if self.port.container.level == 0:
                break 

            # Load components onto vessel's deck
            while self.container.level < self.container.capacity:
                load = LoadComponent(duration=self.load_length.rvs(),
                                      **self.__dict__)
                yield env.process(load.execute())

                if self.port.container.level == 0:
                    break
            
            # Navigate from port to site
            sail = Navigate(duration=self.sail_length.rvs(), 
                            description='Navigate from port to site',
                            **self.__dict__)
            yield env.process(sail.execute())

            # Install component
            install = InstallComponent(duration=self.install_length.rvs(),
                                       **self.__dict__)
            yield env.process(install.execute())

            # Sail to the next site if components still on deck
            while self.container.level > 0:
                
                sail = Navigate(duration=self.sail_between_length.rvs(),
                                description='Navigate to the next site',
                                **self.__dict__)
                yield env.process(sail.execute())

                install = InstallComponent(duration=self.install_length.rvs(),
                                           **self.__dict__)
                yield env.process(install.execute())

            # Navigate from site to port
            sail = Navigate(duration=self.sail_length.rvs(), 
                            description='Navigate from site to port',
                            **self.__dict__)
            yield env.process(sail.execute())



# ----------------------------------------------------------------------------!
class Navigate(object):
    def __init__(self, env: simpy.Environment, log: EventLog, ID: str, 
                 port: Site, container: simpy.Container, owf: Site,
                 duration: float, description='Navigate', **kwargs):
        
        self.env = env
        self.log = log
        self.ID = ID
        self.duration = duration
        self.description = description
        self.port = port
        self.container = container
        self.owf = owf

    def execute(self):
        self.log.entry(activity_state='Initiated', **self.__dict__)
        yield self.env.timeout(self.duration)
        self.log.entry(activity_state='Completed', **self.__dict__)


# ----------------------------------------------------------------------------!
class LoadComponent(object):
    def __init__(self, env: simpy.Environment, log: EventLog, ID: str, 
                 port: Site, container: simpy.Container, owf: Site, 
                 duration: float, description='Load components', **kwargs):

        self.env = env
        self.log = log
        self.ID = ID
        self.duration = duration
        self.description = description
        self.port = port
        self.container = container
        self.owf = owf

    def execute(self):
        self.log.entry(activity_state='Initiated', **self.__dict__)
        yield self.env.timeout(self.duration)

        yield self.port.container.get(1)
        yield self.container.put(1)

        self.log.entry(activity_state='Completed', **self.__dict__)


# ----------------------------------------------------------------------------!
class InstallComponent(object):
    def __init__(self, env: simpy.Environment, log: EventLog, ID: str, 
                 port: Site, container: simpy.Container, owf: Site, 
                 duration: float, description='Install component', **kwargs):

        self.env = env
        self.log = log
        self.ID = ID
        self.duration = duration
        self.description = description
        self.port = port
        self.container = container
        self.owf = owf

    def execute(self):
        self.log.entry(activity_state='Initiated', **self.__dict__)
        yield self.env.timeout(self.duration)
        yield self.container.get(1)
        yield self.owf.container.put(1)
        self.log.entry(activity_state='Completed', **self.__dict__)


# ----------------------------------------------------------------------------!
class Site(object):
    def __init__(self, env: simpy.Environment, log: EventLog, ID: str, 
                 resources: int = 1, capacity: int = 1, level: int = 0,
                 **kwargs):

        self.env = env
        self.log = log
        self.ID = ID
        self.resources = simpy.Resource(env=env, capacity=resources)
        self.container = simpy.Container(env=env, capacity=capacity, init=level)


# ----------------------------------------------------------------------------!
class DiscreteEventSimulation(object):
    def __init__(self, start_date: dt = dt(2021, 1, 1), **kwargs):
        
        self.start_date = start_date
        self.env = simpy.Environment(initial_time=start_date.timestamp())
        self.log = EventLog(**self.__dict__)
        self.entities = self.create_entities()

    def create_entities(self):
        self.port = Site(ID='Saint Brieuc Port', capacity=54, level=54, 
                         **self.__dict__)
        self.owf = Site(ID='Saint Brieuc OWF', capacity=54, level=0, 
                         **self.__dict__)

        self.aeolus = Vessel(ID='Aeolus', **self.__dict__)
        return locals()

    def start_simulation(self):
        for obj in Vessel.instances:
            self.env.process(obj.execute_actions())

        self.env.run()

        Vessel.instances = []
        
    def get_event_log(self):
        return pd.DataFrame(self.log.log)

    def get_project_length(self):
        log_file = self.log.log
        period = log_file['Timestamp'][-1] - log_file['Timestamp'][0]
        period = period.total_seconds() / (24 * 3600) 
        return period

    def empty_log(self):
        self.log = EventLog(**self.__dict__)

In [305]:
periods = []

for i in range(300):
    SaintBrieuc = DiscreteEventSimulation()
    SaintBrieuc.start_simulation()
    periods.append(SaintBrieuc.get_project_length())

periods = np.array(periods)

In [439]:
import plotly.graph_objects as go

# Fit the lognormal distribution
mean, sigma = (periods.mean(), np.std(periods))
periods_fit = lognorm(mean, sigma)
upper = periods_fit.ppf(0.995)
lower = periods_fit.ppf(0.005)
expected_time = np.linspace(lower, upper, 100)
cdf = periods_fit.cdf(t)

# Histogram object
hist = go.Histogram(
    x=periods,
    xbins={'size': 0.025},
    histnorm='probability',
    cumulative={'enabled': True},
    marker={'color': 'red', 'opacity': 0.75, 'line': {'color': 'black', 'width': 1}},
    name='Cumulative distribution'
)

dist = go.Scatter(
    x=expected_time,
    y= 1 - cdf,
    marker={'color': 'blue'},
    name='Exceedance probability'
)

fig = go.Figure(data=[hist, dist])
fig.update_layout(
    title='Expected construction period after 300 iterations',
    xaxis_title='Expected construction period [days]',
    yaxis_title='Probability [-]',
    plot_bgcolor='white',
    xaxis=dict(ticks='inside'),
    yaxis=dict(ticks='inside', range=[0, 1])
)
fig.show()